## Naive Bayes Classifier

In [12]:
import pandas as pd

# import data
df = pd.read_csv('../data/churn.txt', delimiter=',')

In [13]:
df.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.


For the sake of this demonstration, we will only be using Int'l Plan and VMail Plan.

In [17]:
df2 = df[['Int\'l Plan', 'VMail Plan', 'Churn?']]
df2.head()

,Int'l Plan,VMail Plan,Churn?
0,no,yes,False.
1,no,yes,False.
2,no,no,False.
3,yes,no,False.
4,yes,no,False.


In [18]:
# rename columns
df2.rename(columns= 
           {'Int\'l Plan': 'International Plan',
            'VMail Plan': 'Voicemail Plan',
            'Churn?': 'Churn'}, inplace=True)

C:\Users\ryans\AppData\Local\Temp\ipykernel_11940\848690299.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.rename(columns=


In [19]:
df2.head()

,International Plan,Voicemail Plan,Churn
0,no,yes,False.
1,no,yes,False.
2,no,no,False.
3,yes,no,False.
4,yes,no,False.


In [20]:
df2['Churn'] = df2['Churn'].map({'False.': 0,
                                 'True.': 1})

C:\Users\ryans\AppData\Local\Temp\ipykernel_11940\305127014.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Churn'] = df2['Churn'].map({'False.': 0,


In [21]:
df2

,International Plan,Voicemail Plan,Churn
0,no,yes,0
1,no,yes,0
2,no,no,0
3,yes,no,0
4,yes,no,0
...,...,...,...
3328,no,yes,0
3329,no,no,0
3330,no,no,0
3331,yes,no,0


##### Value Counts to Calculate Probabilities

In [22]:
# churn value counts
df2['Churn'].value_counts()

Churn
0    2850
1     483
Name: count, dtype: int64

In [23]:
# international plan value counts
df2['International Plan'].value_counts()

International Plan
no     3010
yes     323
Name: count, dtype: int64

In [24]:
# voicemail plan value counts
df2['Voicemail Plan'].value_counts()

Voicemail Plan
no     2411
yes     922
Name: count, dtype: int64

In [102]:
# calculate probability of churning for entire dataset
churn_probability = 483 / (3333)
churn_probability

0.14491449144914492

In [103]:
# calculate the probability of someone having the voicemail plan (regardless of churn)
voicemail_probability = 922 / 3333
voicemail_probability

0.27662766276627665

In [104]:
# calculate the probability of someone having the international plan (regardless of churn)
international_probability = 323 / 3333
international_probability

0.0969096909690969

In [107]:
not_churn = 1 - churn_probability
not_voicemail = 1 - voicemail_probability
not_international = 1 - international_probability

In [108]:
# data frame of prior probabilities
priors = pd.DataFrame({
    'Churn Prob': churn_probability,
    'Voicemail Prob': voicemail_probability,
    'International Prob': international_probability,
    'Not Churn': not_churn,
    'Not Voicemail': not_voicemail,
    'Not International': not_international
}, index = [0])

priors

,Churn Prob,Voicemail Prob,International Prob,Not Churn,Not Voicemail,Not International
0,0.144914,0.276628,0.09691,0.855086,0.723372,0.90309


Now we will find conditional probabilities starting with the international plan given Churn = False and the voicemail plan given churn = False. The first step is to filter down to churn = False, then from there count the number of individuals who have the international plan.

In [48]:
# international plan yes, churn 0
IP_and_churn_false = df2.loc[(df2['International Plan'] == 'yes') & (df2['Churn'] == 0)].shape[0]
i_c_no = round(IP_and_churn_false / 2850, 4)
print(f'The probability of someone who has churned and has the international plan is {i_c_no}.')

The probability of someone who has churned and has the international plan is 0.0653.


In [64]:
# voicemail plan yes, churn 0
v_and_churn_false = df2.loc[(df2['Voicemail Plan'] == 'yes') & (df2['Churn'] == 0)].shape[0]
v_c_no = round(v_and_churn_false / 2850, 4)
print(f'The probability of someone who has not churned and has the international plan is {v_c_no}.')

The probability of someone who has not churned and has the international plan is 0.2954.


In [109]:
# international plan yes, churn yes (1)
ip_churn_true = df2[(df2['International Plan'] == 'yes') & (df2['Churn'] == 1)].shape[0]
IP_and_churn = ip_churn_true/483

# 483 people have churned and 137 have the plan
print(f'The probability that someone that churned also had the international plan is {round(IP_and_churn, 4)}')

The probability that someone that churned also had the international plan is 0.2836


In [110]:
# voicemail yes, churn 1
vm_churn_true = df2[(df2['Voicemail Plan'] == 'yes') & (df2['Churn'] == 1)].shape[0]
VM_and_churn= vm_churn_true/483

# 80 people have out of the 438 that churned
print(f' The probability that someone who churned had the voicemail plan is {round(VM_and_churn, 4)}')

 The probability that someone who churned had the voicemail plan is 0.1656


#### Bayes Theorem

##### Classifying Those with Both Plans

Now we can find the probability that someone churned given they had the voicemail plan, and the probability that someone churned given they had the international plan. Using the Bayes Theorem we only need to multiply the p(International Plan | Churn) by p(Voicemail Plan | Churn) and the p(Churn = True). 

In [117]:
# probability of international plan given churn times voicemail plan given churn
both = (round(IP_and_churn * VM_and_churn, 4))
churn_posterior = round(both * churn_probability, 4)
print(f'The likelihood that someone with both plans will churn is {churn_posterior}.')

The likelihood that someone with both plans will churn is 0.0068.


In [132]:
# both plans and not churn
both = i_c_no * v_c_no
not_churn_posterior = round((both * not_churn), 4)
print(f'The likelihood that a customer with both plans will not churn is {not_churn_posterior}')

The likelihood that a customer with both plans will not churn is 0.0165


Since the likelihood that a customer with both plans will not churn is higher than a customer that will churn with both plans, the classifer will classify this customer as not churning.

##### Classifying Those with International Plan but no Voicemail Plan

To find the posterior probability of this customer we only need to find the probability of someone not having the voicemail plan who has churned. This is very easy to do since we already have the probability of someone who DOES have the voicemail plan who has churned so we just subtract that number from 1 for p(not V | churn). Then, we multiply this number by p(International Plan = yes | Churn = 1) which we also have calculated. This product is multiplied by the prevelence of churning which was again also found.

In [140]:
# not voicemail who have churned
not_voicemail_churn = 1 - VM_and_churn
not_voicemail_churn

0.834368530020704

In [141]:
# implement bayes formula
both = not_voicemail_churn * IP_and_churn
no_voicemail_yes_ip_churn = round((both * churn_probability), 4)
print(f'The likelihood that a person will churn given international plan but no voicemail is {no_voicemail_yes_ip_churn}')

The likelihood that a person will churn given international plan but no voicemail is 0.0343


In [142]:
# p(not voicemail | churn = 0)
not_voicemail_not_churn = 1 - v_c_no
not_voicemail_not_churn

0.7046

In [144]:
# bayes theory
both = not_voicemail_not_churn * i_c_no
not_voicemail_ip_yes_no_churn = round((both * not_churn), 4)
print(f'The likelihood that a person will not churn given no voicemail plan but yes to international plan is {not_voicemail_ip_yes_no_churn}.')

The likelihood that a person will not churn given no voicemail plan but yes to international plan is 0.0393.


In [145]:
both

0.04601038